In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk import sent_tokenize
from prepare import prepare

In [2]:
with open('data.json') as json_file:
    data = json.load(json_file)

In [3]:
df = pd.DataFrame(data)

In [4]:
df = prepare(df)

In [5]:
df.head()

,repo,language,readme_contents,clean,stemmed,lemmatized
0,iterativv/NostalgiaForInfinity,Python,# NostalgiaForInfinity\nTrading strategy for t...,nostalgiaforinfinity trading strategy freqtrad...,nostalgiaforinfin trade strategi freqtradehttp...,nostalgiaforinfinity trading strategy freqtrad...
1,cl2333/Grokking-the-Coding-Interview-Patterns-...,Python,# [Grokking-the-Coding-Interview-Patterns-for-...,grokkingthecodinginterviewpatternsforcodingque...,grokkingthecodinginterviewpatternsforcodingque...,grokkingthecodinginterviewpatternsforcodingque...
2,edeng23/binance-trade-bot,Python,# binance-trade-bot\n\n![github](https://img.s...,binancetradebot githubhttpsimgshieldsiogithubw...,binancetradebot githubhttpsimgshieldsiogithubw...,binancetradebot githubhttpsimgshieldsiogithubw...
3,donnemartin/system-design-primer,Python,*[English](README.md) ∙ [日本語](README-ja.md) ∙ ...,englishreadmemd readmejamd readmezhhansmd read...,englishreadmemd readmejamd readmezhhansmd read...,englishreadmemd readmejamd readmezhhansmd read...
4,dortania/OpenCore-Legacy-Patcher,Python,"<div align=""center"">\n <img src=""i...",div aligncenter img srcimagesocpatcherpng alto...,div aligncent img srcimagesocpatcherpng altope...,div aligncenter img srcimagesocpatcherpng alto...


# Exploration

In [6]:
def clean(text):
    'A simple function to cleanup text data'
    
    ADDITIONAL_STOPWORDS = []
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [7]:
def show_counts_and_ratios(df, column):
    """
    Takes in a dataframe and a string of a single column
    Returns a dataframe with absolute value counts and percentage value counts
    """
    labels = pd.concat([df[column].value_counts(),
                    df[column].value_counts(normalize=True)], axis=1)
    labels.columns = ['n', 'percent']
    labels
    return labels

show_counts_and_ratios(df, "language")

,n,percent
HTML,29,0.239669
C#,27,0.223140
Python,25,0.206612
JavaScript,23,0.190083
Java,15,0.123967
Ruby,1,0.008264
Shell,1,0.008264


In [11]:
python_words = clean(' '.join(df[df.language == 'Python'].readme_contents))

In [12]:
c_sharp_words = clean(' '.join(df[df.language == 'C#'].readme_contents))

In [13]:
html_words = clean(' '.join(df[df.language == 'HTML'].readme_contents))

In [14]:
javascript_words = clean(' '.join(df[df.language == 'JavaScript'].readme_contents))

In [15]:
java_words = clean(' '.join(df[df.language == 'Java'].readme_contents))

In [16]:
ruby_words = clean(' '.join(df[df.language == 'Ruby'].readme_contents))

In [17]:
shell_words = clean(' '.join(df[df.language == 'Shell'].readme_contents))

### Freq

In [19]:
python_freq = pd.Series(python_words).value_counts()

In [20]:
c_sharp_freq = pd.Series(c_sharp_words).value_counts()

In [21]:
html_freq = pd.Series(html_words).value_counts()

In [22]:
javascript_freq = pd.Series(javascript_words).value_counts()

In [23]:
java_freq = pd.Series(java_words).value_counts()

In [24]:
ruby_freq = pd.Series(ruby_words).value_counts()

In [25]:
shell_freq = pd.Series(shell_words).value_counts()

In [26]:
python_freq.head()

yes        1615
unknown     909
apikey      558
video       489
python      447
dtype: int64

In [27]:
c_sharp_freq.head()

new       192
c         188
use       186
var       185
string    156
dtype: int64

In [30]:
html_freq.head()

html        97
file        76
use         76
15001700    62
dom         51
dtype: int64

In [29]:
javascript_freq.head()

javascript    309
function      276
const         261
1             240
bad           205
dtype: int64

In [31]:
java_freq.head()

aligncentera    189
td              189
width100px      189
altbr           189
java            177
dtype: int64

### Ruby and Shell only have one observation so we can either drop or go find more

In [32]:
ruby_freq.head()

cookbook    6
chef        5
sou         2
backer      2
amazon      2
dtype: int64

In [33]:
shell_freq.head()

java              13
docker            10
run                7
openjdk            5
dockerfilejava     5
dtype: int64